# Proyecto Final - ANALITICA BIG DATA - Modelo predicción lluvia Australia 

### Realizado por: ARANZA GARCÍA BORONAT

In [97]:
# import necessary libraries
import pandas as pd 
import numpy
import matplotlib.pyplot as plt 
from pyspark.sql import SparkSession
# create sparksession
spark = SparkSession \
.builder \
.appName("Pysparkexample") \
.config("spark.some.config.option", "some-value") \
.getOrCreate()

In [98]:
spark.version

'3.1.1'

### Lectura de dataframe

In [99]:
df = spark\
.read\
.format("csv")\
.option("header", True)\
.option("delimiter", ",")\
.option("inferSchema", True)\
.load("rain_tomorrow_in_australia_mod.csv")

In [100]:
df.columns

['Date',
 'Location',
 'MinTemp',
 'MaxTemp',
 'Rainfall',
 'Evaporation',
 'Sunshine',
 'WindGustDir',
 'WindGustSpeed',
 'WindDir9am',
 'WindDir3pm',
 'WindSpeed9am',
 'WindSpeed3pm',
 'Humidity9am',
 'Humidity3pm',
 'Pressure9am',
 'Pressure3pm',
 'Cloud9am',
 'Cloud3pm',
 'Temp9am',
 'Temp3pm',
 'RainToday',
 'RISK_MM',
 'RainTomorrow',
 'PXW']

Ver el total de columnas y tipos

In [101]:
df.printSchema()

root
 |-- Date: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- MinTemp: double (nullable = true)
 |-- MaxTemp: double (nullable = true)
 |-- Rainfall: double (nullable = true)
 |-- Evaporation: double (nullable = true)
 |-- Sunshine: double (nullable = true)
 |-- WindGustDir: string (nullable = true)
 |-- WindGustSpeed: double (nullable = true)
 |-- WindDir9am: string (nullable = true)
 |-- WindDir3pm: string (nullable = true)
 |-- WindSpeed9am: double (nullable = true)
 |-- WindSpeed3pm: double (nullable = true)
 |-- Humidity9am: double (nullable = true)
 |-- Humidity3pm: double (nullable = true)
 |-- Pressure9am: double (nullable = true)
 |-- Pressure3pm: double (nullable = true)
 |-- Cloud9am: double (nullable = true)
 |-- Cloud3pm: double (nullable = true)
 |-- Temp9am: double (nullable = true)
 |-- Temp3pm: double (nullable = true)
 |-- RainToday: string (nullable = true)
 |-- RISK_MM: double (nullable = true)
 |-- RainTomorrow: string (nullable = true)
 |--

Ver el total de registros así como comprobar que no hay duplicados

In [102]:
df.count()

142193

In [103]:
df.distinct().count()

142193

In [104]:
df.show(5)

+----------+--------+-------+-------+--------+-----------+--------+-----------+-------------+----------+----------+------------+------------+-----------+-----------+-----------+-----------+--------+--------+-------+-------+---------+-------+------------+----+
|      Date|Location|MinTemp|MaxTemp|Rainfall|Evaporation|Sunshine|WindGustDir|WindGustSpeed|WindDir9am|WindDir3pm|WindSpeed9am|WindSpeed3pm|Humidity9am|Humidity3pm|Pressure9am|Pressure3pm|Cloud9am|Cloud3pm|Temp9am|Temp3pm|RainToday|RISK_MM|RainTomorrow| PXW|
+----------+--------+-------+-------+--------+-----------+--------+-----------+-------------+----------+----------+------------+------------+-----------+-----------+-----------+-----------+--------+--------+-------+-------+---------+-------+------------+----+
|2008-12-01|  Albury|   13.4|   22.9|     0.6|       null|    null|          W|         44.0|         W|       WNW|        20.0|        24.0|       71.0|       22.0|     1007.7|     1007.1|     8.0|    null|   16.9|   21

Realizo un describe que ya me permite ver el número de nulos por columnas, valores medios, desviación típica, mínimos y máximos, con el fin de tener una visión general de los datos.

In [105]:
df.describe("MinTemp", "MaxTemp","Rainfall", "Evaporation", "Sunshine","WindGustSpeed").show()

+-------+-----------------+-----------------+------------------+-----------------+------------------+------------------+
|summary|          MinTemp|          MaxTemp|          Rainfall|      Evaporation|          Sunshine|     WindGustSpeed|
+-------+-----------------+-----------------+------------------+-----------------+------------------+------------------+
|  count|           141552|           141867|            140787|            81350|             74377|            132919|
|   mean|12.18742158358772|23.23730959278764|2.3499740743107385|5.469824216349125| 7.624853113193599|39.984103100384445|
| stddev|6.412181910203625|7.941855439811297| 8.465172917616467|4.188536508895161|3.7815249942144544|13.588649294637493|
|    min|             -8.5|             -4.8|               0.0|              0.0|               0.0|               6.0|
|    max|            139.2|           1333.4|             371.0|            145.0|              14.5|             135.0|
+-------+-----------------+-----

In [106]:
df.describe('WindSpeed9am',
 'WindSpeed3pm',
 'Humidity9am',
 'Humidity3pm',
 'Pressure9am',
 'Pressure3pm').show()

+-------+-----------------+-----------------+-----------------+------------------+------------------+------------------+
|summary|     WindSpeed9am|     WindSpeed3pm|      Humidity9am|       Humidity3pm|       Pressure9am|       Pressure3pm|
+-------+-----------------+-----------------+-----------------+------------------+------------------+------------------+
|  count|           140845|           139563|           140419|            138583|            128179|            128212|
|   mean|  14.001988000994|18.63757586179718| 68.8438103105705|51.482606091656265|1017.6537584159634|1015.2582035378888|
| stddev|8.893337098234488|8.803345036235546|19.05129253533625|20.797771843698854|  7.10547571152079| 7.036676783493588|
|    min|              0.0|              0.0|              0.0|               0.0|             980.5|             977.1|
|    max|            130.0|             87.0|            100.0|             100.0|            1041.0|            1039.6|
+-------+-----------------+-----

In [107]:
df.describe('Cloud9am',
 'Cloud3pm',
 'Temp9am',
 'Temp3pm',
 'RISK_MM',
 'PXW').show()

+-------+-----------------+-----------------+------------------+-----------------+------------------+------------------+
|summary|         Cloud9am|         Cloud3pm|           Temp9am|          Temp3pm|           RISK_MM|               PXW|
+-------+-----------------+-----------------+------------------+-----------------+------------------+------------------+
|  count|            88536|            85099|            141289|           139467|            142193|            142193|
|   mean|4.437189391885787|4.503166899728551|16.987508581701338|21.68723497314774|2.3606816087992066|12.400000000006994|
| stddev|  2.8870155257336|2.720632530403663|  6.49283832547889| 6.93759386853373|  8.47796906922771|               0.0|
|    min|              0.0|              0.0|              -7.2|             -5.4|               0.0|              12.4|
|    max|              9.0|              9.0|              40.2|             46.7|             371.0|              12.4|
+-------+-----------------+-----

Eliminamos tal como se indican en las instrucciones de entrada las siguientes columnas: "RISK_MM", "Date" and "Location".

In [108]:
from pyspark.sql.functions import col
df = df.drop(col('Date')).drop(col('Location')).drop(col('RISK_MM'))
df.printSchema()

root
 |-- MinTemp: double (nullable = true)
 |-- MaxTemp: double (nullable = true)
 |-- Rainfall: double (nullable = true)
 |-- Evaporation: double (nullable = true)
 |-- Sunshine: double (nullable = true)
 |-- WindGustDir: string (nullable = true)
 |-- WindGustSpeed: double (nullable = true)
 |-- WindDir9am: string (nullable = true)
 |-- WindDir3pm: string (nullable = true)
 |-- WindSpeed9am: double (nullable = true)
 |-- WindSpeed3pm: double (nullable = true)
 |-- Humidity9am: double (nullable = true)
 |-- Humidity3pm: double (nullable = true)
 |-- Pressure9am: double (nullable = true)
 |-- Pressure3pm: double (nullable = true)
 |-- Cloud9am: double (nullable = true)
 |-- Cloud3pm: double (nullable = true)
 |-- Temp9am: double (nullable = true)
 |-- Temp3pm: double (nullable = true)
 |-- RainToday: string (nullable = true)
 |-- RainTomorrow: string (nullable = true)
 |-- PXW: double (nullable = true)



Aplico función para ver el número de nulos de las diferentes columnas

In [109]:
from pyspark.sql.functions import sum as sql_sum, col

all_cols=df.columns

null_counts = [sql_sum(col(c).isNull().cast("int")).alias(c + "_nulls") for c in all_cols]

df.select(*null_counts).show()

+-------------+-------------+--------------+-----------------+--------------+-----------------+-------------------+----------------+----------------+------------------+------------------+-----------------+-----------------+-----------------+-----------------+--------------+--------------+-------------+-------------+---------------+------------------+---------+
|MinTemp_nulls|MaxTemp_nulls|Rainfall_nulls|Evaporation_nulls|Sunshine_nulls|WindGustDir_nulls|WindGustSpeed_nulls|WindDir9am_nulls|WindDir3pm_nulls|WindSpeed9am_nulls|WindSpeed3pm_nulls|Humidity9am_nulls|Humidity3pm_nulls|Pressure9am_nulls|Pressure3pm_nulls|Cloud9am_nulls|Cloud3pm_nulls|Temp9am_nulls|Temp3pm_nulls|RainToday_nulls|RainTomorrow_nulls|PXW_nulls|
+-------------+-------------+--------------+-----------------+--------------+-----------------+-------------------+----------------+----------------+------------------+------------------+-----------------+-----------------+-----------------+-----------------+--------------+

Defino función para sustituir los nulos por las medias de cada columna a excepción de las columnas tipo String que realizaré en el siguiente paso

In [110]:
from pyspark.sql.functions import avg

def fill_with_mean(df, exclude=set()): 
    stats = df.agg(*(
        avg(c).alias(c) for c in df.columns if c not in exclude
    ))
    return df.na.fill(stats.first().asDict())

df = fill_with_mean(df, ["WindGustDir", "WindDir9am", "WindDir3pm", "RainToday", "RainTomorrow"])

In [111]:
df.show(10)

+-------+-------+--------+-----------------+-----------------+-----------+-------------+----------+----------+------------+------------+-----------+-----------+-----------+-----------+-----------------+-----------------+-------+-------+---------+------------+----+
|MinTemp|MaxTemp|Rainfall|      Evaporation|         Sunshine|WindGustDir|WindGustSpeed|WindDir9am|WindDir3pm|WindSpeed9am|WindSpeed3pm|Humidity9am|Humidity3pm|Pressure9am|Pressure3pm|         Cloud9am|         Cloud3pm|Temp9am|Temp3pm|RainToday|RainTomorrow| PXW|
+-------+-------+--------+-----------------+-----------------+-----------+-------------+----------+----------+------------+------------+-----------+-----------+-----------+-----------+-----------------+-----------------+-------+-------+---------+------------+----+
|   13.4|   22.9|     0.6|5.469824216349125|7.624853113193599|          W|         44.0|         W|       WNW|        20.0|        24.0|       71.0|       22.0|     1007.7|     1007.1|              8.0|4.5

Por último en esta preparación y limpieza de datos sustituyo en las variables tipo String los valores vacíos por "unknown"

In [112]:
df = df.na.fill(value="unknown",subset=["WindGustDir", "WindDir9am", "WindDir3pm", "RainToday", "RainTomorrow"])

Compruebo que no quedan nulos

In [113]:
df.printSchema()

root
 |-- MinTemp: double (nullable = false)
 |-- MaxTemp: double (nullable = false)
 |-- Rainfall: double (nullable = false)
 |-- Evaporation: double (nullable = false)
 |-- Sunshine: double (nullable = false)
 |-- WindGustDir: string (nullable = false)
 |-- WindGustSpeed: double (nullable = false)
 |-- WindDir9am: string (nullable = false)
 |-- WindDir3pm: string (nullable = false)
 |-- WindSpeed9am: double (nullable = false)
 |-- WindSpeed3pm: double (nullable = false)
 |-- Humidity9am: double (nullable = false)
 |-- Humidity3pm: double (nullable = false)
 |-- Pressure9am: double (nullable = false)
 |-- Pressure3pm: double (nullable = false)
 |-- Cloud9am: double (nullable = false)
 |-- Cloud3pm: double (nullable = false)
 |-- Temp9am: double (nullable = false)
 |-- Temp3pm: double (nullable = false)
 |-- RainToday: string (nullable = false)
 |-- RainTomorrow: string (nullable = false)
 |-- PXW: double (nullable = false)



In [114]:
from pyspark.sql.functions import count

df.groupBy('RainTomorrow').agg(count('*')).show()

+------------+--------+
|RainTomorrow|count(1)|
+------------+--------+
|          No|  110316|
|         Yes|   31877|
+------------+--------+



Preparación del dataframe

In [115]:
#Vamos a utilizar "filters methods" es el más sencillo, al tener 21 variables tenemos un espacio de dimensión 21, por lo que vamos
# a intentar reducir variables con un "AUC", área bajo la curva ROC, nos quedamos con las que mayor capacidad predictiva tenga.
#AUC como de bueno es un modelo para clasificación binaria, por lo que vamos a resolver el problema una a una variable y veo
#las que mejor AUC me da para así hacer selcción de características. AUC: TPR vs FPR (Sensibilidad vs 1 - Especificidad)
#AUC=P(fa|1 > fa|0), AUC ha de ser próximo a 1. Si tengo 0,5 es totalmente aleatorio se solapan 0 y 1 totalmente, si es menor que 0,5, es mejor darle la vuelta al clasificador.

def get_auc(df, c):

    from pyspark.mllib.evaluation import BinaryClassificationMetrics

    preds_and_labels = df\
    .select(c, 'RainTomorrow')\
    .withColumn('label', when(col('RainTomorrow')=='Yes', 1.0).otherwise(0.0))\
    .rdd\
    .map(lambda r: (r[c], r['label']))

    auc = BinaryClassificationMetrics(preds_and_labels).areaUnderROC

    print(str(auc))

    return (c, auc)

In [116]:
double_cols = [e[0] for e in df.dtypes if e[1]=="double"]

In [117]:
from pyspark.sql.functions import col, when, desc

stylepredcapdf = spark.createDataFrame([get_auc(df, c) for c in double_cols], ['feat', 'auc'])



from pyspark.sql.functions import desc

stylepredcapdf.orderBy(desc('auc')).show()

0.5527071867293107
0.39020806285961396
0.6998593428598958
0.4368569559180371
0.29881251958436095
0.6414502213760991
0.5559240717710611
0.5520068301838192
0.6832462038176417
0.7929080299419458
0.34735920011459714
0.3605525326740113
0.667928573412419
0.6975133431976344
0.47912771555904243
0.3683303549481391
0.5
+-------------+-------------------+
|         feat|                auc|
+-------------+-------------------+
|  Humidity3pm| 0.7929080299419458|
|     Rainfall| 0.6998593428598958|
|     Cloud3pm| 0.6975133431976344|
|  Humidity9am| 0.6832462038176417|
|     Cloud9am|  0.667928573412419|
|WindGustSpeed| 0.6414502213760991|
| WindSpeed9am| 0.5559240717710611|
|      MinTemp| 0.5527071867293107|
| WindSpeed3pm| 0.5520068301838192|
|          PXW|                0.5|
|      Temp9am|0.47912771555904243|
|  Evaporation| 0.4368569559180371|
|      MaxTemp|0.39020806285961396|
|      Temp3pm| 0.3683303549481391|
|  Pressure3pm| 0.3605525326740113|
|  Pressure9am|0.34735920011459714|
|    

Con este dataframe ya vemos que la variable Humidity3pm con un 0.79 de AUC puede ser de gran ayuda en el modelo predictivo, las variables con AUC de 0.5 son como si fueran totalmente aleatorias, (como lanzar una modena al aire), y en el caso de las más bajitas hay que obtener la inversa para que nos sean de utilidad. 

In [118]:
feat_subset = stylepredcapdf\
.withColumn('auc', when(col('auc') < 0.5, 1 - col('auc')).otherwise(col('auc')))\
.orderBy(desc('auc')).show()

+-------------+------------------+
|         feat|               auc|
+-------------+------------------+
|  Humidity3pm|0.7929080299419458|
|     Sunshine| 0.701187480415639|
|     Rainfall|0.6998593428598958|
|     Cloud3pm|0.6975133431976344|
|  Humidity9am|0.6832462038176417|
|     Cloud9am| 0.667928573412419|
|  Pressure9am|0.6526407998854029|
|WindGustSpeed|0.6414502213760991|
|  Pressure3pm|0.6394474673259887|
|      Temp3pm|0.6316696450518609|
|      MaxTemp| 0.609791937140386|
|  Evaporation|0.5631430440819629|
| WindSpeed9am|0.5559240717710611|
|      MinTemp|0.5527071867293107|
| WindSpeed3pm|0.5520068301838192|
|      Temp9am|0.5208722844409576|
|          PXW|               0.5|
+-------------+------------------+



En nuestro caso nos vamos a quedar con las variables de AUC superior a 0.65, si cogieramos mñas de 0,75 tan solo construiriamos el modelo con una variable

In [119]:
feat_subset = stylepredcapdf\
.withColumn('auc', when(col('auc') < 0.5, 1 - col('auc')).otherwise(col('auc')))\
.orderBy(desc('auc')).filter(col('auc') >= 0.65).select('feat')\
.rdd\
.map(lambda r: r['feat']).collect()

In [120]:
feat_subset

['Humidity3pm',
 'Sunshine',
 'Rainfall',
 'Cloud3pm',
 'Humidity9am',
 'Cloud9am',
 'Pressure9am']

# Parte 1 Ejercicio - Crear Modelo Predictivo para toda Australia

# 1º Modelo con Random Forest Classifier

In [25]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.sql.functions import sum as sql_sum, col, when, count

In [26]:
df.printSchema()

root
 |-- MinTemp: double (nullable = false)
 |-- MaxTemp: double (nullable = false)
 |-- Rainfall: double (nullable = false)
 |-- Evaporation: double (nullable = false)
 |-- Sunshine: double (nullable = false)
 |-- WindGustDir: string (nullable = false)
 |-- WindGustSpeed: double (nullable = false)
 |-- WindDir9am: string (nullable = false)
 |-- WindDir3pm: string (nullable = false)
 |-- WindSpeed9am: double (nullable = false)
 |-- WindSpeed3pm: double (nullable = false)
 |-- Humidity9am: double (nullable = false)
 |-- Humidity3pm: double (nullable = false)
 |-- Pressure9am: double (nullable = false)
 |-- Pressure3pm: double (nullable = false)
 |-- Cloud9am: double (nullable = false)
 |-- Cloud3pm: double (nullable = false)
 |-- Temp9am: double (nullable = false)
 |-- Temp3pm: double (nullable = false)
 |-- RainToday: string (nullable = false)
 |-- RainTomorrow: string (nullable = false)
 |-- PXW: double (nullable = false)



In [27]:
df.count()

142193

In [28]:
df.groupBy('RainTomorrow').agg(count('*')).show()

+------------+--------+
|RainTomorrow|count(1)|
+------------+--------+
|          No|  110316|
|         Yes|   31877|
+------------+--------+



Hacemos las partición de los datos

In [29]:
(tr_set, tt_set) = df.randomSplit([0.7, 0.3])


print(tr_set.count())


print(tt_set.count())

99599
42594


Si debemos completar los nulos lo hacemos sobre el conjunto de train.
Ahora vamos a diseñar nuestras etapas del pipeline:
1ª Assembler
2ª Scaler, si elegimos un algoritmo que lo necesita, si es un RF o arbol de decisión no hace falta.
2ª stringindexer =>df=df.withColumn('label_idx', when(col('label')=='famele'
3ª RandomForest

In [30]:
feats = [c[0] for c in df.dtypes if c[1]=='double']

assembler = VectorAssembler()\
.setInputCols(feats)\
.setOutputCol("features")

# Defining the label/target column

label_indexer = StringIndexer()\
.setInputCol("RainTomorrow")\
.setOutputCol("label_idx")

# A random forest as classifier

classstage = RandomForestClassifier()\
    .setNumTrees(50)\
    .setFeatureSubsetStrategy("sqrt")\
    .setImpurity("gini")\
    .setMaxDepth(16)\
    .setMaxBins(32)\
    .setLabelCol("label_idx")\
    .setFeaturesCol("features")\
    .setSubsamplingRate(0.7)\
    .setMinInstancesPerNode(10)

In [31]:
# Defining the pipeline

pipeline = Pipeline()\
    .setStages([assembler, label_indexer, classstage])

 # Training
model1 = pipeline.fit(tr_set)

In [32]:
model1

PipelineModel_15b5d71ebff0

In [33]:
  # Making preds on test data

ttpreds = model1.transform(tt_set)

trpreds = model1.transform(tr_set)

In [34]:
# Evaluating the model

# Model evaluator

evaluator = BinaryClassificationEvaluator()\
.setLabelCol("label_idx")\
.setRawPredictionCol("rawPrediction")

ttauc = evaluator.evaluate(ttpreds)

trauc = evaluator.evaluate(trpreds)

print("\n[Info MyVoiceDetectionApp] ModelModelo con Random Forest Classifier - AUC(tr) = " + str(trauc) + " - AUC(tt) = " + str(ttauc) + "\n")


[Info MyVoiceDetectionApp] ModelModelo con Random Forest Classifier - AUC(tr) = 0.9161648220063296 - AUC(tt) = 0.8739650875499182



####  El mejor resultado de los diferentes modelos realizados, en  test AUC de 0.874

### Realizo el mismo modelo Random Forest Classifier, pero en este caso no uso todas las variables doubles sino las que tienen un mayor AUC, calculadas en la reduccción de variable con método "filters methods" 

In [121]:
assembler = VectorAssembler()\
.setInputCols(feat_subset)\
.setOutputCol("features")

# Defining the label/target column

label_indexer = StringIndexer()\
.setInputCol("RainTomorrow")\
.setOutputCol("label_idx")

# A random forest as classifier

classstage = RandomForestClassifier()\
    .setNumTrees(50)\
    .setFeatureSubsetStrategy("sqrt")\
    .setImpurity("gini")\
    .setMaxDepth(16)\
    .setMaxBins(32)\
    .setLabelCol("label_idx")\
    .setFeaturesCol("features")\
    .setSubsamplingRate(0.7)\
    .setMinInstancesPerNode(10)

In [122]:
# Defining the pipeline

pipeline = Pipeline()\
    .setStages([assembler, label_indexer, classstage])

 # Training
model_R = pipeline.fit(tr_set)

In [124]:
  # Making preds on test data

ttpreds = model_R.transform(tt_set)

trpreds = model_R.transform(tr_set)

In [125]:
# Evaluating the model

# Model evaluator

evaluator = BinaryClassificationEvaluator()\
.setLabelCol("label_idx")\
.setRawPredictionCol("rawPrediction")

ttauc = evaluator.evaluate(ttpreds)

trauc = evaluator.evaluate(trpreds)

print("\n[Info MyVoiceDetectionApp] ModelModelo con Random Forest Classifier - AUC(tr) = " + str(trauc) + " - AUC(tt) = " + str(ttauc) + "\n")


[Info MyVoiceDetectionApp] ModelModelo con Random Forest Classifier - AUC(tr) = 0.8844168724176003 - AUC(tt) = 0.853709958721402



####  Empeoro el resultado en test, por lo que es mejor tener en cuenta todas las varibles. En este caso no son muchas, hay otros casos en los que se ha de construir el modelo con un gran número de variables y el método "filters methods" puede ser de gran ayuda.

# 2º Modelo con Gradient-Boosted Tree Classifier

Aplico el modelo Gadient-Booster Tree Classifier para ver si mejoro el resultado

In [35]:
df.printSchema()

root
 |-- MinTemp: double (nullable = false)
 |-- MaxTemp: double (nullable = false)
 |-- Rainfall: double (nullable = false)
 |-- Evaporation: double (nullable = false)
 |-- Sunshine: double (nullable = false)
 |-- WindGustDir: string (nullable = false)
 |-- WindGustSpeed: double (nullable = false)
 |-- WindDir9am: string (nullable = false)
 |-- WindDir3pm: string (nullable = false)
 |-- WindSpeed9am: double (nullable = false)
 |-- WindSpeed3pm: double (nullable = false)
 |-- Humidity9am: double (nullable = false)
 |-- Humidity3pm: double (nullable = false)
 |-- Pressure9am: double (nullable = false)
 |-- Pressure3pm: double (nullable = false)
 |-- Cloud9am: double (nullable = false)
 |-- Cloud3pm: double (nullable = false)
 |-- Temp9am: double (nullable = false)
 |-- Temp3pm: double (nullable = false)
 |-- RainToday: string (nullable = false)
 |-- RainTomorrow: string (nullable = false)
 |-- PXW: double (nullable = false)



In [126]:
(tr_set, tt_set) = df.randomSplit([0.7, 0.3])


print(tr_set.count())


print(tt_set.count())

99208
42985


In [127]:
from pyspark.ml.classification import GBTClassifier

feats = [c[0] for c in df.dtypes if c[1]=='double']

assembler = VectorAssembler()\
.setInputCols(feats)\
.setOutputCol("features")

# Defining the label/target column

label_indexer = StringIndexer()\
.setInputCol("RainTomorrow")\
.setOutputCol("label_idx")

# Gradient-Boosted Tree Classifier

gbt = GBTClassifier(labelCol="label_idx", featuresCol="features", maxIter=10)


In [128]:
# Defining the pipeline

pipeline2 = Pipeline()\
    .setStages([assembler, label_indexer, gbt])

 # Training
model2 = pipeline2.fit(tr_set)

In [129]:
  # Making preds on test data

ttpreds = model2.transform(tt_set)

trpreds = model2.transform(tr_set)

In [130]:
# Evaluating the model

# Model evaluator

evaluator = BinaryClassificationEvaluator()\
.setLabelCol("label_idx")\
.setRawPredictionCol("rawPrediction")

ttauc = evaluator.evaluate(ttpreds)

trauc = evaluator.evaluate(trpreds)

print("\n[Info MyRainPrediction] Model Gradient-Boosted Tree Classifier - AUC(tr) = " + str(trauc) + " - AUC(tt) = " + str(ttauc) + "\n")


[Info MyRainPrediction] Model Gradient-Boosted Tree Classifier - AUC(tr) = 0.8612484076650002 - AUC(tt) = 0.8599367982351445



#### No mejoro el resultado de Random Forest Classifier con todas las variables doubles

# 3º Modelo con Logistic Regression

Aplico el modelo Logistic Regression para ver si mejoro el resultado

In [41]:
(tr_set, tt_set) = df.randomSplit([0.7, 0.3])


print(tr_set.count())


print(tt_set.count())

99349
42844


In [42]:
from pyspark.ml.classification import LogisticRegression

feats = [c[0] for c in df.dtypes if c[1]=='double']

assembler = VectorAssembler()\
.setInputCols(feats)\
.setOutputCol("features")

# Defining the label/target column

label_indexer = StringIndexer()\
.setInputCol("RainTomorrow")\
.setOutputCol("label_idx")

# Logistic Regression Model

lr = LogisticRegression(labelCol="label_idx", featuresCol="features", maxIter=10,regParam=0.3)

In [43]:
# Defining the pipeline

pipeline3 = Pipeline()\
    .setStages([assembler, label_indexer, lr])

 # Training
model3 = pipeline3.fit(tr_set)

In [44]:
  # Making preds on test data

ttpreds = model3.transform(tt_set)

trpreds = model3.transform(tr_set)

In [45]:
# Evaluating the model

# Model evaluator

evaluator = BinaryClassificationEvaluator()\
.setLabelCol("label_idx")\
.setRawPredictionCol("rawPrediction")

ttauc = evaluator.evaluate(ttpreds)

trauc = evaluator.evaluate(trpreds)

print("\n[Info MyRainPrediction] Model Logistic Regression - AUC(tr) = " + str(trauc) + " - AUC(tt) = " + str(ttauc) + "\n")


[Info MyRainPrediction] Model Logistic Regression - AUC(tr) = 0.8211747079997799 - AUC(tt) = 0.8181858238623891



# Parte 2 del Ejercicio: Modelo de predicción para Canberra

Lectura del daset

In [59]:
df2 = spark\
.read\
.format("csv")\
.option("header", True)\
.option("delimiter", ",")\
.option("inferSchema", True)\
.load("rain_tomorrow_in_australia_mod.csv")

In [60]:
df2.printSchema()

root
 |-- Date: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- MinTemp: double (nullable = true)
 |-- MaxTemp: double (nullable = true)
 |-- Rainfall: double (nullable = true)
 |-- Evaporation: double (nullable = true)
 |-- Sunshine: double (nullable = true)
 |-- WindGustDir: string (nullable = true)
 |-- WindGustSpeed: double (nullable = true)
 |-- WindDir9am: string (nullable = true)
 |-- WindDir3pm: string (nullable = true)
 |-- WindSpeed9am: double (nullable = true)
 |-- WindSpeed3pm: double (nullable = true)
 |-- Humidity9am: double (nullable = true)
 |-- Humidity3pm: double (nullable = true)
 |-- Pressure9am: double (nullable = true)
 |-- Pressure3pm: double (nullable = true)
 |-- Cloud9am: double (nullable = true)
 |-- Cloud3pm: double (nullable = true)
 |-- Temp9am: double (nullable = true)
 |-- Temp3pm: double (nullable = true)
 |-- RainToday: string (nullable = true)
 |-- RISK_MM: double (nullable = true)
 |-- RainTomorrow: string (nullable = true)
 |--

In [61]:
df2.select('Location').distinct().collect()

[Row(Location='Cairns'),
 Row(Location='NorfolkIsland'),
 Row(Location='Bendigo'),
 Row(Location='Walpole'),
 Row(Location='Canberra'),
 Row(Location='Woomera'),
 Row(Location='Adelaide'),
 Row(Location='Cobar'),
 Row(Location='SydneyAirport'),
 Row(Location='Goulburn'),
 Row(Location='PerthAirport'),
 Row(Location='Wollongong'),
 Row(Location='Williamtown'),
 Row(Location='Moree'),
 Row(Location='Mildura'),
 Row(Location='Portland'),
 Row(Location='Albany'),
 Row(Location='SalmonGums'),
 Row(Location='Brisbane'),
 Row(Location='Sydney'),
 Row(Location='Perth'),
 Row(Location='Sale'),
 Row(Location='BadgerysCreek'),
 Row(Location='Hobart'),
 Row(Location='Tuggeranong'),
 Row(Location='Ballarat'),
 Row(Location='GoldCoast'),
 Row(Location='MelbourneAirport'),
 Row(Location='Dartmoor'),
 Row(Location='Nhil'),
 Row(Location='PearceRAAF'),
 Row(Location='Albury'),
 Row(Location='Yass'),
 Row(Location='Witchcliffe'),
 Row(Location='WaggaWagga'),
 Row(Location='Queanbeyan'),
 Row(Location='D

Filtramos las localizaciones que nos interesan para realizar el modelo de Canberra (Tal como se nos indicó, al tener pocos registros Canberra hemos de coger también los de Yass y Queanbeyan)

In [63]:
df3= df2.filter( (df2.Location  == "Canberra") | (df2.Location  == "Yass") | (df2.Location  == "Queanbeyan") )

In [64]:
from pyspark.sql.functions import count

df3.groupBy('Location').agg(count('*')).show()

+----------+--------+
|  Location|count(1)|
+----------+--------+
|  Canberra|      68|
|      Yass|     500|
|Queanbeyan|    2100|
+----------+--------+



####  Realizamos la limpieza del dataset:
    - Eliminamos las columnas que nos se han de tener en cuenta en el modelo
    - Rellenamos los nulos de las columnas double con la media
    - Rellenamos los nulos de las columnas string con "unknown"

In [67]:
from pyspark.sql.functions import col
df3 = df3.drop(col('Date')).drop(col('Location')).drop(col('RISK_MM'))
df3.printSchema()

root
 |-- MinTemp: double (nullable = true)
 |-- MaxTemp: double (nullable = true)
 |-- Rainfall: double (nullable = true)
 |-- Evaporation: double (nullable = true)
 |-- Sunshine: double (nullable = true)
 |-- WindGustDir: string (nullable = true)
 |-- WindGustSpeed: double (nullable = true)
 |-- WindDir9am: string (nullable = true)
 |-- WindDir3pm: string (nullable = true)
 |-- WindSpeed9am: double (nullable = true)
 |-- WindSpeed3pm: double (nullable = true)
 |-- Humidity9am: double (nullable = true)
 |-- Humidity3pm: double (nullable = true)
 |-- Pressure9am: double (nullable = true)
 |-- Pressure3pm: double (nullable = true)
 |-- Cloud9am: double (nullable = true)
 |-- Cloud3pm: double (nullable = true)
 |-- Temp9am: double (nullable = true)
 |-- Temp3pm: double (nullable = true)
 |-- RainToday: string (nullable = true)
 |-- RainTomorrow: string (nullable = true)
 |-- PXW: double (nullable = true)



In [68]:
df3 = fill_with_mean(df3, ["WindGustDir", "WindDir9am", "WindDir3pm", "RainToday", "RainTomorrow"])

In [69]:
df3 = df3.na.fill(value="unknown",subset=["WindGustDir", "WindDir9am", "WindDir3pm", "RainToday", "RainTomorrow"])

Comprobamos que en todas las variables los nulos = false

In [70]:
df3.printSchema()

root
 |-- MinTemp: double (nullable = false)
 |-- MaxTemp: double (nullable = false)
 |-- Rainfall: double (nullable = false)
 |-- Evaporation: double (nullable = false)
 |-- Sunshine: double (nullable = false)
 |-- WindGustDir: string (nullable = false)
 |-- WindGustSpeed: double (nullable = false)
 |-- WindDir9am: string (nullable = false)
 |-- WindDir3pm: string (nullable = false)
 |-- WindSpeed9am: double (nullable = false)
 |-- WindSpeed3pm: double (nullable = false)
 |-- Humidity9am: double (nullable = false)
 |-- Humidity3pm: double (nullable = false)
 |-- Pressure9am: double (nullable = false)
 |-- Pressure3pm: double (nullable = false)
 |-- Cloud9am: double (nullable = false)
 |-- Cloud3pm: double (nullable = false)
 |-- Temp9am: double (nullable = false)
 |-- Temp3pm: double (nullable = false)
 |-- RainToday: string (nullable = false)
 |-- RainTomorrow: string (nullable = false)
 |-- PXW: double (nullable = false)



### Realizamos en Modelo de Random Forest para la zona de Canberra

In [72]:
(tr_set_C, tt_set_C) = df3.randomSplit([0.7, 0.3])


print(tr_set_C.count())


print(tt_set_C.count())

1872
796


In [78]:
tr_set_C.groupBy('RainTomorrow').agg(count('*')).show()

+------------+--------+
|RainTomorrow|count(1)|
+------------+--------+
|          No|    1551|
|         Yes|     321|
+------------+--------+



In [79]:
tt_set_C.groupBy('RainTomorrow').agg(count('*')).show()

+------------+--------+
|RainTomorrow|count(1)|
+------------+--------+
|          No|     638|
|         Yes|     158|
+------------+--------+



In [92]:
feats1 = [c[0] for c in df3.dtypes if c[1]=='double']

assembler_C = VectorAssembler()\
.setInputCols(feats1)\
.setOutputCol("features_C")

# Defining the label/target column

label_indexer_C = StringIndexer()\
.setInputCol("RainTomorrow")\
.setOutputCol("label_idx_C")

# A random forest as classifier

classstage_C = RandomForestClassifier()\
    .setNumTrees(50)\
    .setFeatureSubsetStrategy("sqrt")\
    .setImpurity("gini")\
    .setMaxDepth(16)\
    .setMaxBins(32)\
    .setLabelCol("label_idx_C")\
    .setFeaturesCol("features_C")\
    .setSubsamplingRate(0.7)\
    .setMinInstancesPerNode(10)

In [93]:
# Defining the pipeline

pipeline = Pipeline()\
    .setStages([assembler_C, label_indexer_C, classstage_C])

 # Training
modelC = pipeline.fit(tr_set_C)

In [94]:
 # Making preds on test data

ttpreds_C = modelC.transform(tt_set_C)

trpreds_C = modelC.transform(tr_set_C)

In [95]:
# Evaluating the model

# Model evaluator

evaluator_C = BinaryClassificationEvaluator()\
.setLabelCol("label_idx_C")\
.setRawPredictionCol("rawPrediction")

ttauc_C = evaluator_C.evaluate(ttpreds_C)

trauc_C = evaluator_C.evaluate(trpreds_C)

print("\n[Info MyVoiceDetectionApp] ModelModelo con Random Forest Classifier Canberra - AUC(tr) = " + str(trauc_C) + " - AUC(tt) = " + str(ttauc_C) + "\n")


[Info MyVoiceDetectionApp] ModelModelo con Random Forest Classifier Canberra - AUC(tr) = 0.9263524085556303 - AUC(tt) = 0.8611860640450779



#### En este caso aunque tenemos un punto de partida con pocos datos conseguimos un AUC en TEST de 0,86

Para cualquier consulta adicional mis contactos son: aranzazu.garcia@live.u-tad.com y también: aranza9991@gmail.com